In [281]:
# Isaac Berez
# 17.01.23
import sys
from scipy.io import mmread
import os
import glob
import pandas as pd
import numpy as np
#from pandas_ods_reader import read_ods
from copy import deepcopy
import pprint
import json
import re
from datetime import datetime
import logging
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.cluster import DBSCAN
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage, optimal_leaf_ordering, leaves_list
import harmonypy as hm
from matplotlib.cm import ScalarMappable
from datetime import date
import mpld3
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn
import bokeh
from bokeh.resources import INLINE
from adjustText import adjust_text
from scipy.stats import mannwhitneyu, false_discovery_control, wilcoxon
import matplotlib as mpl




import dimorph_processing as dp
import cell_comparison as cc
import sex_stats as ss
import sig_gene_analysis as sga
import run_sig_genes as rsg

today = str(date.today())
%matplotlib notebook
%load_ext autoreload
%autoreload 2

#change matplotlib font type to make compatibile with illustrator
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# GABA

In [259]:
#get stat sig genes based on following criteria
log_fc = np.log2(1.5) #(log2(1.5))
alpha = 0.05

#run using run_sig_genes.py
gaba_all_genes_folder = '/bigdata/isaac/gaba_files/sex_stats/311224_run/volcano_plots/'
gaba_sex_stats_folder = '/bigdata/isaac/gaba_files/sex_stats/311224_run/'
gaba_sig_gene_heatmap_folder = '/bigdata/isaac/gaba_files/sex_stats/311224_run/sig_gene_heatmaps/'
gaba_ct_bar_plots_folder = '/bigdata/isaac/gaba_files/sex_stats/311224_run/ct_bar_plots/'

gaba_sig_deltas, gaba_sig_p_adj, gaba_sig_genes, gaba_dimorph_cell_types_df_sorted = rsg.run_sig_genes(gaba_all_genes_folder, gaba_sex_stats_folder, gaba_sig_gene_heatmap_folder, gaba_ct_bar_plots_folder, cell_class='gaba', log_fc = np.log2(1.5),alpha = 0.05, savefig = True, write_to_file = False)


/home/isaac/analysis/scRNA-seq/run_sig_genes.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fn = np.array(metadata_df_dlr.loc['full_name'][np.sort(idx)])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [143]:
# Check for duplicate rows ignoring the left-most column
duplicates = gaba_sig_genes.iloc[:, :].duplicated(keep=False)

# Print duplicate rows if any
if duplicates.any():
    print("Duplicate rows found:")
    print(gaba_sig_genes[duplicates])
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [144]:
gaba_sdimorphic_cts = []
for fn in gaba_dimorph_cell_types_df_sorted.index:
    #if more than have than stat sig gene counts come from either mf delta, cell type is sexually dimorphic
    if gaba_dimorph_cell_types_df_sorted.loc[fn,'gene_count']/2 < gaba_dimorph_cell_types_df_sorted.loc[fn,'Δmf_B'] + gaba_dimorph_cell_types_df_sorted.loc[fn,'Δmf_N']:
        gaba_sdimorphic_cts.append(fn)
gaba_sdimorphic_cts

['GABA-33-Cartpt-Unc13c', 'GABA-24-Prlr-St18']

In [145]:
gaba_partially_sdimorphic_cts = []
mu_mfb = np.mean(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)
two_sigma_mfb = 2*np.std(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)
mu_mfn = np.mean(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)
two_sigma_mfn = 2*np.std(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)
for fn in gaba_dimorph_cell_types_df_sorted.index:
    if gaba_dimorph_cell_types_df_sorted.loc[fn,'Δmf_B'] > mu_mfb+two_sigma_mfb or gaba_dimorph_cell_types_df_sorted.loc[fn,'Δmf_N'] > mu_mfn+two_sigma_mfn:
        gaba_partially_sdimorphic_cts.append(fn)
gaba_partially_sdimorphic_cts

['GABA-33-Cartpt-Unc13c', 'GABA-27-Greb1-Dlk1', 'GABA-24-Prlr-St18']

In [295]:
#investigate gene composition of GABA-33-Cartpt-Unc13c
cluster_fn = 'GABA-33-Cartpt-Unc13c'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(gaba_sig_p_adj.loc[:,gaba_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   gaba_sig_deltas.loc[:,gaba_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
#subset = ['Cntn5','Esr1','Kcnab1','Plxnab2']
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn +'_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           gaba_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_gaba_33 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_gaba_33['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_gaba_33['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [241]:
tmp_df_gaba_33

,gene,cluster_fn
0,Kcnab1,GABA-33-Cartpt-Unc13c
1,Plxna2,GABA-33-Cartpt-Unc13c


In [294]:
#investigate gene composition of GABA-24-Prlr-St18
cluster_fn = 'GABA-24-Prlr-St18'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(gaba_sig_p_adj.loc[:,gaba_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   gaba_sig_deltas.loc[:,gaba_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           gaba_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_gaba_24 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_gaba_24['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_gaba_24['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [239]:
tmp_df_gaba_24

,gene,cluster_fn
0,Brinp2,GABA-24-Prlr-St18
1,Cartpt,GABA-24-Prlr-St18
2,Hpcal1,GABA-24-Prlr-St18
3,Rcn1,GABA-24-Prlr-St18


In [304]:

cluster_fn = 'GABA-27-Greb1-Dlk1'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(gaba_sig_p_adj.loc[:,gaba_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   gaba_sig_deltas.loc[:,gaba_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)
#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           gaba_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              29,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo_first29',
                            savefig = True)

tmp_df_gaba_27 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_gaba_27['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_gaba_27['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [111]:
cluster_fn = 'GABA-26-Col18a1-Jsrp1'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(gaba_sig_p_adj.loc[:,gaba_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   gaba_sig_deltas.loc[:,gaba_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              gaba_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = True)

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  d_rows,d_cols = np.where(np.abs(sig_deltas.iloc[:subset,:])>0.58)


In [83]:
Counter(gaba_sig_genes.loc[gaba_sig_genes.loc[:,'cluster_fn'] == 'GABA-27-Greb1-Dlk1','test'])

Counter({'ΔBN_m': 52, 'Δmf_N': 9, 'ΔBN_f': 2, 'Δmf_B': 2})

In [116]:
np.std(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)

2.6197820569962174

In [129]:
np.mean(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0) + 2*np.std(gaba_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)

8.60848787783749

# Vglut1

In [303]:
#get stat sig genes based on following criteria
log_fc = np.log2(1.5) #(log2(1.5))
alpha = 0.05

#run using run_sig_genes.py
Vglut1_all_genes_folder = '/bigdata/isaac/Vglut1_files/sex_stats/311224_run/volcano_plots/'
Vglut1_sex_stats_folder = '/bigdata/isaac/Vglut1_files/sex_stats/311224_run/'
Vglut1_sig_gene_heatmap_folder = '/bigdata/isaac/Vglut1_files/sex_stats/311224_run/sig_gene_heatmaps/'
Vglut1_ct_bar_plots_folder = '/bigdata/isaac/Vglut1_files/sex_stats/311224_run/ct_bar_plots/'

Vglut1_sig_deltas, Vglut1_sig_p_adj, Vglut1_sig_genes, Vglut1_dimorph_cell_types_df_sorted = rsg.run_sig_genes(Vglut1_all_genes_folder, Vglut1_sex_stats_folder, Vglut1_sig_gene_heatmap_folder, Vglut1_ct_bar_plots_folder, cell_class='Vglut1', log_fc = np.log2(1.5),alpha = 0.05, savefig = True, write_to_file = False)


/home/isaac/analysis/scRNA-seq/run_sig_genes.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fn = np.array(metadata_df_dlr.loc['full_name'][np.sort(idx)])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [151]:

duplicates = Vglut1_sig_genes.iloc[:, :].duplicated(keep=False)

# Print duplicate rows if any
if duplicates.any():
    print("Duplicate rows found:")
    print(Vglut1_sig_genes[duplicates])
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [152]:
Vglut1_sdimorphic_cts = []
for fn in Vglut1_dimorph_cell_types_df_sorted.index:
    #if more than half than stat sig gene counts come from either mf delta, cell type is sexually dimorphic
    if Vglut1_dimorph_cell_types_df_sorted.loc[fn,'gene_count']/2 < Vglut1_dimorph_cell_types_df_sorted.loc[fn,'Δmf_B'] + Vglut1_dimorph_cell_types_df_sorted.loc[fn,'Δmf_N']:
        Vglut1_sdimorphic_cts.append(fn)
Vglut1_sdimorphic_cts

['Vglut1-12-Dlk1-Dcn',
 'Vglut1-10-Ecel1-Sox5',
 'Vglut1-7-Il33-Prox1',
 'Vglut1-4-Snca-Satb2']

In [153]:
Vglut1_partially_sdimorphic_cts = []
mu_mfb = np.mean(Vglut1_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)
two_sigma_mfb = 2*np.std(Vglut1_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)
mu_mfn = np.mean(Vglut1_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)
two_sigma_mfn = 2*np.std(Vglut1_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)
for fn in Vglut1_dimorph_cell_types_df_sorted.index:
    if Vglut1_dimorph_cell_types_df_sorted.loc[fn,'Δmf_B'] > mu_mfb+two_sigma_mfb or Vglut1_dimorph_cell_types_df_sorted.loc[fn,'Δmf_N'] > mu_mfn+two_sigma_mfn:
        Vglut1_partially_sdimorphic_cts.append(fn)
Vglut1_partially_sdimorphic_cts

['Vglut1-10-Ecel1-Sox5', 'Vglut1-4-Snca-Satb2']

In [137]:
Counter(Vglut1_sig_genes.loc[Vglut1_sig_genes.loc[:,'cluster_fn'] == 'Vglut1-7-Il33-Prox1','test'])

Counter({'Δmf_B': 11, 'ΔBN_m': 5, 'Δmf_N': 2, 'ΔBN_f': 1})

In [138]:
Vglut1_dimorph_cell_types_df_sorted

,gene_count,ΔBN_m,ΔBN_f,Δmf_B,Δmf_N,ct_size
cluster_fn,,,,,,
Vglut1-12-Dlk1-Dcn,209,11,66,132,0,673
Vglut1-11-C1ql2-Pcdh11x,18,9,6,2,1,324
Vglut1-9-Slc1a3-Ecm2,5,1,4,0,0,975
Vglut1-8-Fst-Cd44,81,40,3,37,1,525
Vglut1-7-Il33-Prox1,19,5,1,11,2,433
Vglut1-6-Trh-Rxfp1,185,74,62,48,1,339
Vglut1-4-Snca-Satb2,29,0,0,29,0,156
Vglut1-3-Tshz2-Stxbp6,102,89,7,5,1,384
Vglut1-2-Rims3-Cyp26b1,2,1,0,1,0,150


In [296]:
cluster_fn = 'Vglut1-7-Il33-Prox1'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(Vglut1_sig_p_adj.loc[:,Vglut1_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   Vglut1_sig_deltas.loc[:,Vglut1_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           Vglut1_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_vglut1_7 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_vglut1_7['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_vglut1_7['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [234]:
tmp_df_vglut1_7

,gene,cluster_fn
0,Gm10076,Vglut1-7-Il33-Prox1
1,Hmbs,Vglut1-7-Il33-Prox1


In [297]:
cluster_fn = 'Vglut1-12-Dlk1-Dcn'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(Vglut1_sig_p_adj.loc[:,Vglut1_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   Vglut1_sig_deltas.loc[:,Vglut1_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           Vglut1_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_vglut1_12 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_vglut1_12['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_vglut1_12['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [232]:
tmp_df_vglut1_12

,gene,cluster_fn


In [298]:
cluster_fn = 'Vglut1-4-Snca-Satb2'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(Vglut1_sig_p_adj.loc[:,Vglut1_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   Vglut1_sig_deltas.loc[:,Vglut1_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           Vglut1_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_vglut1_4 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_vglut1_4['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_vglut1_4['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [230]:
tmp_df_vglut1_4

,gene,cluster_fn


In [299]:
cluster_fn = 'Vglut1-10-Ecel1-Sox5'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(Vglut1_sig_p_adj.loc[:,Vglut1_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   Vglut1_sig_deltas.loc[:,Vglut1_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           Vglut1_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              Vglut1_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_vglut1_10 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_vglut1_10['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_vglut1_10['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [228]:
tmp_df_vglut1_10

,gene,cluster_fn
0,Ecel1,Vglut1-10-Ecel1-Sox5
1,Gm10076,Vglut1-10-Ecel1-Sox5


# Vglut2

In [302]:
#get stat sig genes based on following criteria
log_fc = np.log2(1.5) #(log2(1.5))
alpha = 0.05

#run using run_sig_genes.py
Vglut2_all_genes_folder = '/bigdata/isaac/Vglut2_files/sex_stats/311224_run/volcano_plots/'
Vglut2_sex_stats_folder = '/bigdata/isaac/Vglut2_files/sex_stats/311224_run/'
Vglut2_sig_gene_heatmap_folder = '/bigdata/isaac/Vglut2_files/sex_stats/311224_run/sig_gene_heatmaps/'
Vglut2_ct_bar_plots_folder = '/bigdata/isaac/Vglut2_files/sex_stats/311224_run/ct_bar_plots/'

Vglut2_sig_deltas, Vglut2_sig_p_adj, Vglut2_sig_genes, Vglut2_dimorph_cell_types_df_sorted = rsg.run_sig_genes(Vglut2_all_genes_folder, Vglut2_sex_stats_folder, Vglut2_sig_gene_heatmap_folder, Vglut2_ct_bar_plots_folder, cell_class='Vglut2', log_fc = np.log2(1.5),alpha = 0.05, savefig = True, write_to_file = False)


/home/isaac/analysis/scRNA-seq/run_sig_genes.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fn = np.array(metadata_df_dlr.loc['full_name'][np.sort(idx)])


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [159]:
duplicates = Vglut2_sig_genes.iloc[:, :].duplicated(keep=False)

# Print duplicate rows if any
if duplicates.any():
    print("Duplicate rows found:")
    print(Vglut2_sig_genes[duplicates])
else:
    print("No duplicate rows found.")

No duplicate rows found.


In [160]:
Vglut2_sdimorphic_cts = []
for fn in Vglut2_dimorph_cell_types_df_sorted.index:
    #if more than half than stat sig gene counts come from either mf delta, cell type is sexually dimorphic
    if Vglut2_dimorph_cell_types_df_sorted.loc[fn,'gene_count']/2 < Vglut2_dimorph_cell_types_df_sorted.loc[fn,'Δmf_B'] + Vglut2_dimorph_cell_types_df_sorted.loc[fn,'Δmf_N']:
        Vglut2_sdimorphic_cts.append(fn)
Vglut2_sdimorphic_cts

['Vglut2-6-Otp-Zic5']

In [161]:
Vglut2_partially_sdimorphic_cts = []
mu_mfb = np.mean(Vglut2_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)
two_sigma_mfb = 2*np.std(Vglut2_dimorph_cell_types_df_sorted.loc[:,'Δmf_B'],axis=0)
mu_mfn = np.mean(Vglut2_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)
two_sigma_mfn = 2*np.std(Vglut2_dimorph_cell_types_df_sorted.loc[:,'Δmf_N'],axis=0)
for fn in Vglut2_dimorph_cell_types_df_sorted.index:
    if Vglut2_dimorph_cell_types_df_sorted.loc[fn,'Δmf_B'] > mu_mfb+two_sigma_mfb or Vglut2_dimorph_cell_types_df_sorted.loc[fn,'Δmf_N'] > mu_mfn+two_sigma_mfn:
        Vglut2_partially_sdimorphic_cts.append(fn)
Vglut2_partially_sdimorphic_cts

['Vglut2-6-Otp-Zic5', 'Vglut2-5-Otp-Peg10']

In [301]:
cluster_fn = 'Vglut2-6-Otp-Zic5'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(Vglut2_sig_p_adj.loc[:,Vglut2_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   Vglut2_sig_deltas.loc[:,Vglut2_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              Vglut2_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           Vglut2_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              Vglut2_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_vglut2_6 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_vglut2_6['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_vglut2_6['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [224]:
tmp_df_vglut2_6

,gene,cluster_fn
0,Gm10076,Vglut2-6-Otp-Zic5
1,Cadps2,Vglut2-6-Otp-Zic5
2,Ecel1,Vglut2-6-Otp-Zic5
3,Greb1,Vglut2-6-Otp-Zic5


In [300]:
cluster_fn = 'Vglut2-5-Otp-Peg10'
max_fc_sorted, sig_deltas_sorted, mask_sorted = sga.p_mask_max_abs(Vglut2_sig_p_adj.loc[:,Vglut2_sig_p_adj.columns.get_level_values(1) == cluster_fn],
                                                                   Vglut2_sig_deltas.loc[:,Vglut2_sig_deltas.columns.get_level_values(1) == cluster_fn])
subset = len(np.where(max_fc_sorted>log_fc)[0])
sga.plot_sig_gene_heatmap_sct(sig_deltas_sorted,
                              mask_sorted, 
                              cluster_fn, 
                              subset,                        
                              Vglut2_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_max_abs_approach_heatmap_q_dot_d_halo',
                            savefig = False)

#linkage sorted version
sig_deltas_subset_linkage, mask_subset_linkage = sga.linkage_sort_sig_genes(cluster_fn,
                           sig_deltas_sorted, 
                           mask_sorted, subset, 
                           Vglut2_sig_gene_heatmap_folder, 
                           str(cluster_fn) + '_linkage_dd', 
                           savefig=False)
sga.plot_sig_gene_heatmap_sct(sig_deltas_subset_linkage,
                              mask_subset_linkage, 
                              cluster_fn, 
                              subset,                        
                              Vglut2_sig_gene_heatmap_folder,
                              cluster_fn + '_p_mask_linkage_heatmap_q_dot_d_halo',
                            savefig = True)

tmp_df_vglut2_5 = pd.DataFrame(columns=['gene', 'cluster_fn'])
tmp_df_vglut2_5['gene'] = mask_sorted.index[np.where(mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_B')] & mask_sorted.loc[:,('p_adj', cluster_fn, 'Δmf_N')] & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_B')] > log_fc) & (sig_deltas_sorted.loc[:,('delta', cluster_fn, 'Δmf_N')] > log_fc))[0]]
tmp_df_vglut2_5['cluster_fn'] = cluster_fn

<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


<IPython.core.display.Javascript object>

/home/isaac/analysis/scRNA-seq/sig_gene_analysis.py:235: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "k." (-> color='k'). The keyword argument will take precedence.
  plt.plot(q_cols+0.5,q_rows+0.5,'k.',markersize = 10,c = 'yellow')


In [237]:
tmp_df_vglut2_5

,gene,cluster_fn
0,Ctnna2,Vglut2-5-Otp-Peg10
1,Ptp4a1,Vglut2-5-Otp-Peg10


In [ ]:
sdegs_df = pd.DataFrame(columns=['gene', 'cluster_fn'])
sdegs_df



,gene,cluster_fn


In [244]:
sdegs_df = pd.concat([tmp_df_vglut2_5,
                      tmp_df_vglut2_6,
                      tmp_df_vglut1_7,
                      tmp_df_vglut1_10,
                      tmp_df_gaba_33,
                      tmp_df_gaba_24,
                      tmp_df_gaba_27], axis = 0)

In [248]:
sdegs_df

,gene,cluster_fn
0,Ctnna2,Vglut2-5-Otp-Peg10
1,Ptp4a1,Vglut2-5-Otp-Peg10
0,Gm10076,Vglut2-6-Otp-Zic5
1,Cadps2,Vglut2-6-Otp-Zic5
2,Ecel1,Vglut2-6-Otp-Zic5
3,Greb1,Vglut2-6-Otp-Zic5
0,Gm10076,Vglut1-7-Il33-Prox1
1,Hmbs,Vglut1-7-Il33-Prox1
0,Ecel1,Vglut1-10-Ecel1-Sox5
1,Gm10076,Vglut1-10-Ecel1-Sox5


In [252]:
full_sdegs_list = [str(x) for x in sdegs_df.loc[:,'gene']]

In [255]:
full_sdegs_list

['Ctnna2',
 'Ptp4a1',
 'Gm10076',
 'Cadps2',
 'Ecel1',
 'Greb1',
 'Gm10076',
 'Hmbs',
 'Ecel1',
 'Gm10076',
 'Kcnab1',
 'Plxna2',
 'Brinp2',
 'Cartpt',
 'Hpcal1',
 'Rcn1',
 'Greb1',
 'Brinp2']

In [166]:
garsetti_sdegs_df = pd.read_excel('/bigdata/isaac/garsetti_sex_genes.xlsx')

In [170]:
garsetti_sdegs_df

,Approved Gene name,HGNC ID,Location,Approved Gene Symbol,Lung1,Heart left ventricle1,Skeletal2 Muscle,Thyroid1,cultured Fibroblasts3,newborn (day 1) and adult vascular endothelial cells4
0,RNA Pol II TRANSCRIPTION INITIATION (Sainsbury...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cyclin H,HGNC:1594,5q14.3,CCNH,"M>F (Naqvi et al., 2019)",NaN,"M>F* (Naqvi et al., 2019)","M>F (Naqvi et al., 2019); F>M* (Lopes-Ramos et...",NaN,NaN
2,cyclin dependent kinase 7,HGNC:1778,5q13.2,CDK7,"M>F (Naqvi et al., 2019)",NaN,"M>F (Naqvi et al., 2019)","F>M (Naqvi et al., 2019); M>F (Lopes-Ramos et ...",NaN,NaN
3,"ERCC excision repair 2, TFIIH core complex hel...",HGNC:3434,19q13.32,ERCC2,"F>M (Naqvi et al., 2019; Oliva et al., 2020)",NaN,NaN,NaN,NaN,NaN
4,general transcription factor IIA subunit 1,HGNC:4646,14q31.1,GTF2A1,NaN,NaN,NaN,NaN,NaN,"M>F (Hartman et al., 2020)"
...,...,...,...,...,...,...,...,...,...,...
75,U2 small nuclear RNA auxiliary factor 1,HGNC:12453,21q22.3,U2AF1,NaN,NaN,"F>M (Lopes-Ramos et al., 2020)",NaN,NaN,NaN
76,U2 small nuclear RNA auxiliary factor 2,HGNC:23156,19q13.42,U2AF2,NaN,NaN,NaN,NaN,NaN,"F>M (Hartman et al., 2020)"
77,zinc finger CCHC-type and RNA binding motif co...,HGNC:29620,12q12,ZCRB1,NaN,NaN,"M>F (Lopes-Ramos et al., 2020)","M>F (Lopes-Ramos et al., 2020)",NaN,NaN
78,zinc finger matrin-type 5,HGNC:28046,22q12.2,ZMAT5,"F>M (Naqvi et al., 2019)",NaN,"F>M* (Naqvi et al., 2019)","F>M* (Naqvi et al., 2019)",NaN,NaN


In [175]:
garsetti_sdegs_df.insert(4, 'gene_name', [str(x).lower().capitalize() for x in garsetti_sdegs_df.loc[:,'Approved Gene Symbol']])

In [254]:
list(garsetti_sdegs_df.loc[:,'gene_name'])

['Nan',
 'Ccnh',
 'Cdk7',
 'Ercc2',
 'Gtf2a1',
 'Gtf2a2',
 'Gtf2f1',
 'Gtf2f2',
 'Gtf2h2',
 'Gtf2h5',
 'Mnat1',
 'Polr2b',
 'Polr2c',
 'Polr2e',
 'Polr2f',
 'Polr2g',
 'Polr2i',
 'Polr2l',
 'Polr2j',
 'Taf1',
 'Taf2',
 'Taf3',
 'Taf5',
 'Taf7',
 'Taf9',
 'Taf10',
 'Taf11',
 'Taf12',
 'Taf13',
 'Tbp',
 'Nan',
 'Nan',
 'Cpsf7',
 'Cstf1',
 'Cstf2',
 'Cstf3',
 'Rbbp6',
 'Nan',
 'Cmtr1',
 'Cmtr2',
 'Rngtt',
 'Rnmt',
 'Nan',
 'Brsk1',
 'Eftud2',
 'Lsm2',
 'Lsm3',
 'Lsm4',
 'Phf5a',
 'Prpf3',
 'Prpf4',
 'Prpf6',
 'Prpf8',
 'Prpf31',
 'Rnpc3',
 'Sart1',
 'Sf1',
 'Sf3a2',
 'Sf3a3',
 'Sf3b2',
 'Sf3b3',
 'Sf3b4',
 'Sf3b5',
 'Sf3b6',
 'Snrnp25',
 'Snrnp35',
 'Snrnp40',
 'Snrnp48',
 'Snrnp70',
 'Snrnp200',
 'Snrpc',
 'Snrpd1',
 'Snrpd2',
 'Snrpe',
 'Txnl4a',
 'U2af1',
 'U2af2',
 'Zcrb1',
 'Zmat5',
 'Zrsr2']

In [251]:
for g in list(garsetti_sdegs_df.loc[:,'gene_name']):
    if g in [str(x) for x in sdegs_df.loc[:,'gene']]:
        print (g)